In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, ServerlessSpec
from DLAIUtils import Utils
import DLAIUtils

import os
import time
import torch
from tqdm.auto import tqdm

In [3]:
dataset = load_dataset("quora", split='train[240000:290000]')
dataset[:5]

Generating train split:   0%|          | 0/404290 [00:00<?, ? examples/s]

{'questions': [{'id': [207550, 351729],
   'text': ['What is the truth of life?', "What's the evil truth of life?"]},
  {'id': [33183, 351730],
   'text': ['Which is the best smartphone under 20K in India?',
    'Which is the best smartphone with in 20k in India?']},
  {'id': [351731, 351732],
   'text': ['Steps taken by Canadian government to improve literacy rate?',
    'Can I send homemade herbal hair oil from India to US via postal or private courier services?']},
  {'id': [37799, 94186],
   'text': ['What is a good way to lose 30 pounds in 2 months?',
    'What can I do to lose 30 pounds in 2 months?']},
  {'id': [351733, 351734],
   'text': ['Which of the following most accurately describes the translation of the graph y = (x+3)^2 -2 to the graph of y = (x -2)^2 +2?',
    'How do you graph x + 2y = -2?']}],
 'is_duplicate': [False, True, False, True, False]}

In [4]:
questions = []
for record in dataset['questions']:
    questions.extend(record['text'])
question = list(set(questions))
print('\n'.join(question[:5]))
print('-'*50)
print(f"Number of questions: {len(question)}")

How does the lie rank in human interaction imperfection?
Why do I feel weak after masturbation?
How do I best way to get over an ex?
What happens after clearing the UPSC examination?
How do you define love in China?
--------------------------------------------------
Number of questions: 88919


In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device != 'cuda':
    print("Sorry, no cuda.")
model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

Sorry, no cuda.


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
query = 'which city is the most populated in the world?'
xq = model.encode(query)
xq.shape

(384,)

# Setup Pinecone

In [8]:
utils = Utils()
PINECONE_API_KEY = utils.get_pinecone_api_key()
pinecone = Pinecone(api_key=PINECONE_API_KEY)
INDEX_NAME = Utils.create_dlai_index_name('dl-ai')

if INDEX_NAME in [index.name for index in pinecone.list_indexes()]:
    pinecone.delete_index(INDEX_NAME)
print(INDEX_NAME)
pinecone.create_index(
    name=INDEX_NAME,
    dimension=model.get_sentence_embedding_dimension(),
    metric="cosine",
    spec=ServerlessSpec(cloud='aws', region='us-west-2')
)
# Things stop working here becuase I don't actually have an OpenAI API key
index = pinecone.Index(INDEX_NAME)
print(index)

PineconeConfigurationError: You haven't specified an Api-Key.

## Create Embeddings and Upsert to Pinecone

In [9]:
batch_size = 200
vector_limt=10000
questions = question[:vector_limt]

import json

for i in tqdm(range(0, len(questiosn), batch_size)):
    i_end = min(i+batch_size, len(questions))                   # End of the batch
    ids = [str(x) for x in range(i, i_end)]                     # Create batch IDs
    metadata = [{'text': text} for text in questions[i:i_end]]  # Create metadata batch
    xc = model.encode([questions[i:i_end]])                     # Create embeddings
    records = zip(ids, xc, metadata)                            # Zip the batch
    index.upsert(vectors=records)                              # Upsert the batch


NameError: name 'questiosn' is not defined

In [ ]:
index.describe_index()

## Run Your Query

In [ ]:
# small helper fucntion so we can repeat queries later
def run_query(query):
    embedding = model.encode(query).tolist()
    results = index.querh(top_k=10, vector=embedding, include_metadata=True, include_values=False)
    for result in results['matches']:
        print(f"{round(result['score'], 2)}: {result['metadata']['text']}")

run_query("Which city has the highest population in the world?")
query = "How do I make chocolate cake?"
run_query(query)